In [1]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import torch.optim as optim

Dataset

In [ ]:
TABULAR_COLS = ["bedrooms","bathrooms","sqft_living","floors","grade"]
TARGET_COL = "log_price"

class HousePriceDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = f"{self.img_dir}/{row['id']}.jpg"
        if not os.path.exists(img_path):
            return None

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        tabular = torch.tensor(row[TABULAR_COLS].values, dtype=torch.float32)
        price = torch.tensor(row[TARGET_COL], dtype=torch.float32)

        return image, tabular, price


Model

In [ ]:
class MultimodalPriceModel(nn.Module):
    def __init__(self, num_tabular):
        super().__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()

        self.tabular_fc = nn.Sequential(
            nn.Linear(num_tabular, 32),
            nn.ReLU()
        )

        self.regressor = nn.Sequential(
            nn.Linear(512 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, img, tab):
        img_feat = self.cnn(img)
        tab_feat = self.tabular_fc(tab)
        return self.regressor(torch.cat([img_feat, tab_feat], 1)).squeeze()


Training

In [ ]:
df = pd.read_csv("data/clean_train.csv")
image_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

dataset = HousePriceDataset(df, "data/images/train", image_transform)
dataset_small = Subset(dataset, range(14000))

loader = DataLoader(dataset_small, batch_size=4, shuffle=True)

model = MultimodalPriceModel(len(TABULAR_COLS))
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(2):
    total = 0
    for img, tab, y in loader:
        optimizer.zero_grad()
        pred = model(img, tab)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total += loss.item()
    print(f"Epoch {epoch+1}, Loss {total/len(loader):.4f}")


In [ ]:
from sklearn.metrics import r2_score

df_train=pd.read_csv('/Users/prashantmaurya/Desktop/Satellite_Property_Valuation/train_actual_vs_predicted.csv')

y_true = df_train["actual_price"].values
y_pred = df_train["predicted_price"].values

r2 = r2_score(y_true, y_pred)
print("R2 score:", r2)
